In [ ]:
import numpy as np
import string
import time
import json
import shutil
import os
import pyspark
import pyspark.sql.functions as F
from datetime import timedelta
from datetime import datetime as dt
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, ArrayType
from IPython.display import clear_output
from delta import *
from faker import Faker

In [ ]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.default.parallelism", "24")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
sc = spark.sparkContext

In [ ]:
class Environment:

    def __init__(self) -> None:
        self.DIR_ROOT = os.getcwd()+"\sx"
        self.DIR_COMPANIES = os.path.join(self.DIR_ROOT, "companies")

In [ ]:
ENV = Environment()

In [ ]:
schema = StructType([
    StructField("ticker", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("timestamp", TimestampType(), True),
])

while True:
    time.sleep(5)
    df = spark.read.json(os.path.join(ENV.DIR_COMPANIES, "*", "trades", "*.json"), schema=schema).orderBy(["ticker", "timestamp"])
    df.write.mode("overwrite").format("delta").save(os.path.join(ENV.DIR_ROOT, "delta","trades"))